In [1]:
# packages
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import glob
from matplotlib.patches import Patch
from matplotlib.lines import Line2D

In [30]:
ng_flow = pd.read_csv(r"../../data/energy_system/fuel_production/resource_flow_ng.csv")
ng_jobs = pd.read_csv(r"../../data/energy_system/nameplate_capacity/per_MWh_om_jobs_ng.csv")
ng_local = pd.read_excel(r"../../data/energy_system/fuel_production/localshare_ng_state.xlsx")
coal_flow = pd.read_csv(r"../../data/energy_system/fuel_production/resource_flow_coal.csv")
coal_jobs = pd.read_csv(r"../../data/energy_system/nameplate_capacity/per_MWh_om_jobs_coal.csv")
coal_local = pd.read_excel(r"../../data/energy_system/fuel_production/localshare_coal_state.xlsx")
state_names = pd.read_excel(r"../../data/demographics/State_Abbr.xlsx")

In [46]:
def fuel_trade_jobs(state,fuel,MWH,year):
    
    """
    state = "AL"
    fuel = "ng-cc"
    MWH = 100
    local_share = 0.2
    year = 2050
    """
    
    full_name = state_names.query("Abbr==@state")["State"].iloc[0]
    if fuel == "coal":
        this_flow = coal_flow
        this_jobs = coal_jobs
        fuelname = "coal"
        local_share = coal_local.query("Abbr ==@state").iloc[0,-1]/100
    else:
        this_flow = ng_flow
        this_jobs = ng_jobs
        fuelname = "ng"
        local_share = ng_local.query("Abbr ==@state").iloc[0,-1]/100
        
    this_state_flow = this_flow.query("END==@full_name")
    this_state_flow = this_state_flow[this_state_flow["START"]!=full_name]
    trade_sum = this_state_flow["TON"].sum()
    this_state_flow["ratio"] = this_state_flow["TON"]/trade_sum
    outsourced = MWH *(1-local_share) # fuel outsourced from other states
    this_state_flow["usage"] = this_state_flow["ratio"]*outsourced # MWh of fuel sourced from this state

    outsource_list = list(this_state_flow["START"])
    if len(outsource_list) == 0: # not on the list of trade states
        all_state_jobs_trade = 0
    else:
        for o_state in outsource_list:
            this_usage = this_state_flow.query("START==@o_state")["usage"].iloc[0]
            ab_o_state = state_names.query("State==@o_state")["Abbr"].iloc[0]
            trade_state_jobs_orig = this_jobs.query("state==@ab_o_state")
            trade_state_jobs_calc = trade_state_jobs_orig.copy()
            trade_state_jobs_calc.iloc[:,1:] = trade_state_jobs_calc.iloc[:,1:]*this_usage
            if o_state == outsource_list[0]:
                all_state_jobs_trade = trade_state_jobs_calc
            else:
                all_state_jobs_trade = all_state_jobs_trade.append(trade_state_jobs_calc)

        all_state_jobs_trade["year"] = year
        all_state_jobs_trade["tech"] = fuelname+"-trade"
        all_state_jobs_trade["type"] = "OM"
    
    # if these job numbers are all zero, then replace the dataframe with zero
    if isinstance(all_state_jobs_trade, pd.DataFrame) and (all_state_jobs_trade['Mining'] == 0).all():
        all_state_jobs_trade = 0
    
    return all_state_jobs_trade